#red base radial

entrenamiento

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
import joblib
import time
import psutil  # Para medir el uso de recursos

# Datos sintéticos: colores ajustados para mejorar la diferenciación
X = np.array([
    # ROJO (alto R, bajo G, bajo B)
    [255, 50, 50],          # rojo claro
    [210, 40, 40],          # rojo medio
    [200, 30, 30],          # rojo suave
    [230, 70, 60],          # rojo brillante
    [220, 60, 50],          # rojo con más naranja
    [235, 20, 20],          # rojo más fuerte
    [240, 10, 10],          # rojo oscuro
    [245, 60, 50],          # rojo cálido
    [230, 50, 40],          # rojo anaranjado
    [250, 30, 20],          # rojo brillante

    # AMARILLO (alto R, alto G, bajo B)
    [255, 255, 20],         # amarillo cálido
    [255, 255, 40],         # amarillo más saturado
    [255, 240, 60],         # amarillo suave
    [255, 230, 80],         # amarillo más verde
    [250, 245, 50],         # amarillo oscuro
    [240, 255, 80],         # amarillo más verde
    [245, 255, 60],         # amarillo con más verde
    [255, 220, 40],         # amarillo más rojo
    [255, 255, 80],         # amarillo más claro
    [250, 230, 20],         # amarillo cálido y profundo

    # VERDE (bajo R, alto G, bajo B)
    [0, 255, 20],           # verde claro
    [10, 230, 20],          # verde medio
    [0, 200, 20],           # verde oscuro
    [30, 240, 30],          # verde brillante
    [10, 210, 10],          # verde suave
    [5, 255, 5],            # verde saturado
    [20, 235, 20],          # verde cálido
    [10, 250, 30],          # verde brillante
    [0, 180, 0],            # verde profundo
    [25, 240, 25],          # verde con más azul
])

# Etiquetas: 0=rojo, 1=amarillo, 2=verde
y = np.array(
    [0]*10 +  # rojo
    [1]*10 +  # amarillo
    [2]*10    # verde
)

# Clustering para obtener los centros RBF
n_centros = 8  # Aumentar el número de centros para mejorar la clasificación
kmeans = KMeans(n_clusters=n_centros, random_state=42).fit(X)
centros = kmeans.cluster_centers_

def rbf(x, centro, sigma=100.0):
    return np.exp(-np.linalg.norm(x - centro)**2 / (2 * sigma**2))

def transformar_con_rbf(X):
    return np.array([[rbf(x, c) for c in centros] for x in X])

# Transformar los datos con la capa RBF
X_rbf = transformar_con_rbf(X)

# Clasificador
modelo = LogisticRegression(max_iter=100000)

# Medir el tiempo de entrenamiento
inicio = time.time()
modelo.fit(X_rbf, y)
fin = time.time()

# Guardar modelo
joblib.dump(modelo, 'entrenamientos_guardados/modelo_rbf.pkl')
joblib.dump(centros, 'entrenamientos_guardados/centros_rbf.pkl')

# Medir el uso de CPU y memoria durante el entrenamiento
cpu_usage = psutil.cpu_percent(interval=1)
memory_usage = psutil.virtual_memory().percent

# Calcular el tiempo de entrenamiento
tiempo_entrenamiento = fin - inicio

# Mostrar los resultados del entrenamiento
print(f"✅ Modelo RBF entrenado con colores más diferenciados.")
print(f"⏱️ Tiempo de Entrenamiento: {tiempo_entrenamiento:.4f} segundos.")
print(f"💻 Uso de CPU durante entrenamiento: {cpu_usage}%")
print(f"💾 Uso de Memoria durante entrenamiento: {memory_usage}%")


ejecucion

In [ ]:
import cv2
import numpy as np
import joblib

# Cargar el modelo entrenado
modelo = joblib.load('entrenamientos_guardados/modelo_rbf.pkl')
centros = joblib.load('entrenamientos_guardados/centros_rbf.pkl')

def rbf(x, centro, sigma=100.0):
    return np.exp(-np.linalg.norm(x - centro)**2 / (2 * sigma**2))

def transformar_con_rbf(X):
    return np.array([[rbf(x, c) for c in centros] for x in X])

def preprocesar_imagen(frame):
    # Convertir la imagen a HSV para hacer la detección más robusta
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Definir los rangos de color de los semáforos en HSV
    rango_rojo_bajo = np.array([0, 100, 100])
    rango_rojo_alto = np.array([10, 255, 255])
    rango_amarillo_bajo = np.array([20, 100, 100])
    rango_amarillo_alto = np.array([40, 255, 255])
    rango_verde_bajo = np.array([40, 100, 100])
    rango_verde_alto = np.array([80, 255, 255])

    # Crear máscaras para cada color
    mascara_rojo = cv2.inRange(hsv, rango_rojo_bajo, rango_rojo_alto)
    mascara_amarillo = cv2.inRange(hsv, rango_amarillo_bajo, rango_amarillo_alto)
    mascara_verde = cv2.inRange(hsv, rango_verde_bajo, rango_verde_alto)

    # Filtrar los colores para obtener solo el predominante
    predom_color = None
    if cv2.countNonZero(mascara_rojo) > max(cv2.countNonZero(mascara_amarillo), cv2.countNonZero(mascara_verde)):
        predom_color = [0]  # Rojo
    elif cv2.countNonZero(mascara_amarillo) > max(cv2.countNonZero(mascara_rojo), cv2.countNonZero(mascara_verde)):
        predom_color = [1]  # Amarillo
    else:
        predom_color = [2]  # Verde

    return predom_color

def detectar_color(frame):
    # Redimensionar para acelerar
    img = cv2.resize(frame, (100, 100))
    # Calcular color promedio
    promedio = img.mean(axis=0).mean(axis=0)  # BGR
    promedio_rgb = promedio[::-1]  # Convertir a RGB
    X_test = transformar_con_rbf([promedio_rgb])
    pred = modelo.predict(X_test)[0]
    return pred

def advertir_color(color_id):
    if color_id == 0:
        return "DETECTADO: ROJO - DETENER"
    elif color_id == 1:
        return "DETECTADO: AMARILLO - PRECAUCION"
    elif color_id == 2:
        return "DETECTADO: VERDE - AVANZAR"
    return "Color no identificado"

def iniciarCamara():
    print("🔧 Iniciando cámara...")
    cam = cv2.VideoCapture(0)
    if not cam.isOpened():
        print("❌ No se pudo abrir la cámara")
        return
    print("✅ Cámara abierta")

    while True:
        ret, frame = cam.read()
        if not ret:
            print("❌ Error al capturar imagen")
            break

        # Preprocesar la imagen para detectar el color predominante
        color_id = preprocesar_imagen(frame)

        mensaje = advertir_color(color_id[0])

        # Mostrar cámara y mensaje
        cv2.putText(frame, mensaje, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.imshow("Camara Semaforo RBF", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # ESC para salir
            break

    cam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    iniciarCamara()


In [ ]:
import cv2
import numpy as np
import joblib
import time
import psutil  # Para medir el uso de recursos

# Cargar el modelo entrenado
modelo = joblib.load('entrenamientos_guardados/modelo_rbf.pkl')
centros = joblib.load('entrenamientos_guardados/centros_rbf.pkl')

def rbf(x, centro, sigma=100.0):
    return np.exp(-np.linalg.norm(x - centro)**2 / (2 * sigma**2))

def transformar_con_rbf(X):
    return np.array([[rbf(x, c) for c in centros] for x in X])

def preprocesar_imagen(frame):
    # Convertir la imagen a HSV para hacer la detección más robusta
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Definir los rangos de color de los semáforos en HSV
    rango_rojo_bajo = np.array([0, 100, 100])
    rango_rojo_alto = np.array([10, 255, 255])
    rango_amarillo_bajo = np.array([20, 100, 100])
    rango_amarillo_alto = np.array([40, 255, 255])
    rango_verde_bajo = np.array([40, 100, 100])
    rango_verde_alto = np.array([80, 255, 255])

    # Crear máscaras para cada color
    mascara_rojo = cv2.inRange(hsv, rango_rojo_bajo, rango_rojo_alto)
    mascara_amarillo = cv2.inRange(hsv, rango_amarillo_bajo, rango_amarillo_alto)
    mascara_verde = cv2.inRange(hsv, rango_verde_bajo, rango_verde_alto)

    # Filtrar los colores para obtener solo el predominante
    predom_color = None
    if cv2.countNonZero(mascara_rojo) > max(cv2.countNonZero(mascara_amarillo), cv2.countNonZero(mascara_verde)):
        predom_color = [0]  # Rojo
    elif cv2.countNonZero(mascara_amarillo) > max(cv2.countNonZero(mascara_rojo), cv2.countNonZero(mascara_verde)):
        predom_color = [1]  # Amarillo
    else:
        predom_color = [2]  # Verde

    return predom_color

def detectar_color(frame):
    # Redimensionar para acelerar
    img = cv2.resize(frame, (100, 100))
    # Calcular color promedio
    promedio = img.mean(axis=0).mean(axis=0)  # BGR
    promedio_rgb = promedio[::-1]  # Convertir a RGB
    X_test = transformar_con_rbf([promedio_rgb])
    pred = modelo.predict(X_test)[0]
    return pred

def advertir_color(color_id):
    if color_id == 0:
        return "DETECTADO: ROJO - DETENER"
    elif color_id == 1:
        return "DETECTADO: AMARILLO - PRECAUCION"
    elif color_id == 2:
        return "DETECTADO: VERDE - AVANZAR"
    return "Color no identificado"

def iniciarCamara():
    print("🔧 Iniciando cámara...")
    cam = cv2.VideoCapture(0)
    if not cam.isOpened():
        print("❌ No se pudo abrir la cámara")
        return
    print("✅ Cámara abierta")

    predicciones_correctas = 0
    total_predicciones = 0
    errores = 0

    while True:
        ret, frame = cam.read()
        if not ret:
            print("❌ Error al capturar imagen")
            break

        # Medir el tiempo de procesamiento por frame
        inicio_frame = time.time()

        # Preprocesar la imagen para detectar el color predominante
        color_id = preprocesar_imagen(frame)

        mensaje = advertir_color(color_id[0])

        # Calcular el color promedio y hacer la predicción
        pred = detectar_color(frame)
        total_predicciones += 1
        if pred == color_id[0]:
            predicciones_correctas += 1
        else:
            errores += 1

        # Mostrar cámara y mensaje
        cv2.putText(frame, mensaje, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.imshow("Camara Semaforo RBF", frame)

        # Medir el tiempo de ejecución por frame
        fin_frame = time.time()
        tiempo_frame = (fin_frame - inicio_frame) * 1000  # En milisegundos

        if cv2.waitKey(1) & 0xFF == 27:  # ESC para salir
            break

    cam.release()
    cv2.destroyAllWindows()

    # Calcular precisión y tasa de error
    precision = predicciones_correctas / total_predicciones
    tasa_error = errores / total_predicciones

    # Medir el uso de CPU y memoria
    cpu_usage = psutil.cpu_percent(interval=1)
    memory_usage = psutil.virtual_memory().percent

    print(f"✅ Evaluación final:")
    print(f"   Precisión: {precision * 100:.2f}%")
    print(f"   Tasa de error: {tasa_error * 100:.2f}%")
    print(f"   Tiempo de ejecución promedio por frame: {tiempo_frame:.2f} ms")
    print(f"   Uso de CPU: {cpu_usage}%")
    print(f"   Uso de memoria: {memory_usage}%")

if __name__ == "__main__":
    iniciarCamara()
